<a href="https://colab.research.google.com/github/nishu204/DM_notebooks/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash
!pip install dash_bootstrap_components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 3.6 MB/s eta 0:00:00


In [5]:
# Logistic Regression function from scratch
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X_train, y_train, X_test, learning_rate=0.01, num_epochs=1000):
    # Initialize weights and bias
    num_features = X_train.shape[1]
    weights = np.zeros(num_features)
    bias = 0

    # Gradient Descent
    for epoch in range(num_epochs):
        # Forward pass
        z = np.dot(X_train, weights) + bias
        y_pred = sigmoid(z)

        # Compute gradients
        dw = (1 / len(y_train)) * np.dot(X_train.T, (y_pred - y_train))
        db = (1 / len(y_train)) * np.sum(y_pred - y_train)

        # Update weights and bias
        weights -= learning_rate * dw
        bias -= learning_rate * db

    # Make predictions on the test set
    z_test = np.dot(X_test, weights) + bias
    y_pred_test = sigmoid(z_test)
    predictions = np.round(y_pred_test)

    return predictions


In [8]:
import dash
from dash import dcc, html, callback
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import numpy as np

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout of the application
app.layout = html.Div([
    html.H1("Logistic Regression Classifier Performance Evaluation"),

    # Dropdown to select dataset
    dcc.Dropdown(
        id='dataset-dropdown',
        options=[
            {'label': 'Iris Dataset', 'value': 'iris'},
            {'label': 'Breast Cancer Dataset', 'value': 'cancer'}
        ],
        value='iris'
    ),

    # Confusion Matrix and Metrics display
    html.Div([
        html.Div([
            html.H3("Confusion Matrix"),
            dcc.Graph(id='confusion-matrix_c')
        ], className='six columns'),

        html.Div([
            html.H3("Performance Metrics"),
            html.Div(id='performance-metrics_c')
        ], className='six columns'),
    ], className='row')
])


# Callback to update confusion matrix and metrics
@callback(
    [Output('confusion-matrix_c', 'figure'),
     Output('performance-metrics_c', 'children')],
    [Input('dataset-dropdown', 'value')]
)
def update_confusion_matrix(selected_dataset):
    # Load the selected dataset
    if selected_dataset == 'iris':
        data = datasets.load_iris()
    else:
        data = datasets.load_breast_cancer()

    X = data.data
    y = data.target

    # Add a column of ones to X for bias term
    X = np.column_stack((np.ones(X.shape[0]), X))

    # Convert labels to binary for logistic regression
    y_binary = (y > np.median(y)).astype(int)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

    # Use the custom Logistic Regression function
    y_pred = logistic_regression(X_train, y_train, X_test)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Calculate performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    misclassification_rate = 1 - accuracy

    # Create confusion matrix figure
    figure = {
        'data': [{
            'z': cm,
            'type': 'heatmap',
            'colorscale': 'Viridis',
            'colorbar': {
                'title': 'Count',
                'titleside': 'right'
            }
        }],
        'layout': {
            'title': 'Confusion Matrix',
            'xaxis': {'title': 'Predicted'},
            'yaxis': {'title': 'True'}
        }
    }

    # Create performance metrics display
    metrics_display = html.Div([
        html.P(f"Recognition Rate: {accuracy * 100:.2f}%"),
        html.P(f"Misclassification Rate: {misclassification_rate * 100:.2f}%"),
        html.P(f"Sensitivity: {sensitivity:.2f}"),
        html.P(f"Specificity: {specificity:.2f}"),
        html.P(f"Precision: {precision:.2f}"),
        html.P(f"Recall: {recall:.2f}")
    ])

    return figure, metrics_display

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>